# Natural Language Processing

Welcome to your natural language processing programming assignment. In this notebook, you will implement a model that classifies the sentiments into positive or negative. You will be able to train the RNN on IMDB dataset.
You will learn to:
Apply an RNN to train on natural language.
Classify the sentiments based on the model trained.


This lab exercise is about a basic form of Natural Language Processing (NLP) called Sentiment Analysis, in which we will try and classify a movie review as either positive or negative.

Consider a simple example: "This movie is not very good." This text ends with the words "very good" which indicates a very positive sentiment, but it is negated because it is preceded by the word "not", so the text should be classified as having a negative sentiment. How can we teach a Neural Network to do this classification?

Another problem is that neural networks cannot work directly on text-data, so we need to convert text into numbers that are compatible with a neural network.

Yet another problem is that a text may be arbitrarily long. Now we need a type of neural network that can work on both short and long sequences of text.


## Recommended Reading

- NLP with python - [Tutorial](https://medium.com/towards-artificial-intelligence/natural-language-processing-nlp-with-python-tutorial-for-beginners-1f54e610a1a0)
- What Is NLP & [How Does it Work?](https://monkeylearn.com/natural-language-processing/)
- Sentiment Analysis (Opinion Mining) with Python — [NLP Tutorial](https://medium.com/towards-artificial-intelligence/sentiment-analysis-opinion-mining-with-python-nlp-tutorial-d1f173ca4e3c)
- Sequence Models [Notes](https://github.com/mbadry1/DeepLearning.ai-Summary/tree/master/5-%20Sequence%20Models#natural-language-processing--word-embeddings)
- [Machine translation with python](https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd)

### NLP libraries
- [NLTK library](https://www.nltk.org)
- [spacy](https://spacy.io/)
- [gensim](https://pypi.org/project/gensim/)
- [pattern](https://github.com/clips/pattern)
- [textblob](https://textblob.readthedocs.io/en/dev/)


Let's get started

First of all, mount google drive.
This will mount the google drive for google colab and you will be able access contents of your drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import os
os.chdir('/content/drive/My Drive/Deep_learning_unit/Assignment11')
folder = os.path.join('/content/drive/My Drive/Deep_learning_unit/Assignment11')
!ls

## 1 - Sentiment Analysis

We need several processing steps for sentiment analysis. First we need to convert the raw text-words into so-called tokens which are integer values. These tokens are really just indices into a list of the entire vocabulary. Then we convert these integer-tokens into so-called embeddings which are real-valued vectors, whose mapping will be trained along with the neural network, so as to map words with similar meanings to similar embedding-vectors. Then we input these embedding-vectors to a Recurrent Neural Network which can take sequences of arbitrary length as input and output a kind of summary of what it has seen in the input. This output is then squashed using a Sigmoid-function to give us a value between 0.0 and 1.0, where 0.0 is taken to mean a negative sentiment and 1.0 means a positive sentiment. This whole process allows us to classify input-text as either having a negative or positive sentiment.

The flowchart of the algorithm is given below:

In [ ]:
from IPython.display import Image 
Image(open('/content/drive/My Drive/Deep_learning_unit/Assignment11/images/flowchart.png','rb').read())

## 2 - Recurrent Neural Network

The basic building block in a Recurrent Neural Network (RNN) is a Recurrent Unit (RU). There are many different variants of recurrent units such as the rather clunky LSTM (Long-Short-Term-Memory) and the somewhat simpler GRU (Gated Recurrent Unit) which we will use in this tutorial. Experiments in the literature suggest that the LSTM and GRU have roughly similar performance. Even simpler variants also exist and the literature suggests that they may perform even better than both LSTM and GRU, but they are not implemented in Keras which we will use in this tutorial.

The following figure shows the abstract idea of a recurrent unit, which has an internal state that is being updated every time the unit receives a new input. This internal state serves as a kind of memory. However, it is not a traditional kind of computer memory which stores bits that are either on or off. Instead the recurrent unit stores floating-point values in its memory-state, which are read and written using matrix-operations so the operations are all differentiable. This means the memory-state can store arbitrary floating-point values (although typically limited between -1.0 and 1.0) and the network can be trained like a normal neural network using Gradient Descent.

The new state-value depends on both the old state-value and the current input. For example, if the state-value has memorized that we have recently seen the word "not" and the current input is "good" then we need to store a new state-value that memorizes "not good" which indicates a negative sentiment.

The part of the recurrent unit that is responsible for mapping old state-values and inputs to the new state-value is called a gate, but it is really just a type of matrix-operation. There is another gate for calculating the output-values of the recurrent unit. The implementation of these gates vary for different types of recurrent units. This figure merely shows the abstract idea of a recurrent unit. The LSTM has more gates than the GRU but some of them are apparently redundant so they can be omitted.

In order to train the recurrent unit, we must gradually change the weight-matrices of the gates so the recurrent unit gives the desired output for an input sequence. This is done automatically in TensorFlow.

In [ ]:
from IPython.display import Image 
Image(open('/content/drive/My Drive/Deep_learning_unit/Assignment11/images/RNN.png','rb').read())

### 2.1 - Unrolled Network

Another way to visualize and understand a Recurrent Neural Network is to "unroll" the recursion. In this figure there is only a single recurrent unit denoted RU, which will receive a text-word from the input sequence in a series of time-steps.

The initial memory-state of the RU is reset to zero internally by Keras / TensorFlow every time a new sequence begins.

In the first time-step the word "this" is input to the RU which uses its internal state (initialized to zero) and its gate to calculate the new state. The RU also uses its other gate to calculate the output but it is ignored here because it is only needed at the end of the sequence to output a kind of summary.

In the second time-step the word "is" is input to the RU which now uses the internal state that was just updated from seeing the previous word "this".

There is not much meaning in the words "this is" so the RU probably doesn't save anything important in its internal state from seeing these words. But when it sees the third word "not" the RU has learned that it may be important for determining the overall sentiment of the input-text, so it needs to be stored in the memory-state of the RU, which can be used later when the RU sees the word "good" in time-step 6.

Finally when the entire sequence has been processed, the RU outputs a vector of values that summarizes what it has seen in the input sequence. We then use a fully-connected layer with a Sigmoid activation to get a single value between 0.0 and 1.0 which we interpret as the sentiment either being negative (values close to 0.0) or positive (values close to 1.0).

Note that for the sake of clarity, this figure doesn't show the mapping from text-words to integer-tokens and embedding-vectors, as well as the fully-connected Sigmoid layer on the output.

In [ ]:
from IPython.display import Image 
Image(open('/content/drive/My Drive/Deep_learning_unit/Assignment11/images/RNN1.png','rb').read())

### 2.2 - 3-Layer Unrolled Network

In this tutorial we will use a Recurrent Neural Network with 3 recurrent units (or layers) denoted RU1, RU2 and RU3 in the "unrolled" figure below.

The first layer is much like the unrolled figure above for a single-layer RNN. First the recurrent unit RU1 has its internal state initialized to zero by Keras / TensorFlow. Then the word "this" is input to RU1 and it updates its internal state. Then it processes the next word "is", and so forth. But instead of outputting a single summary value at the end of the sequence, we use the output of RU1 for every time-step. This creates a new sequence that can then be used as input for the next recurrent unit RU2. The same process is repeated for the second layer and this creates a new output sequence which is then input to the third layer's recurrent unit RU3, whose final output is passed to a fully-connected Sigmoid layer that outputs a value between 0.0 (negative sentiment) and 1.0 (positive sentiment).

Note that for the sake of clarity, the mapping of text-words to integer-tokens and embedding-vectors has been omitted from this figure.

In [ ]:
from IPython.display import Image 
Image(open('/content/drive/My Drive/Deep_learning_unit/Assignment11/images/RNN2.png','rb').read())

### 2.3 - Exploding & Vanishing Gradients

In order to train the weights for the gates inside the recurrent unit, we need to minimize some loss-function which measures the difference between the actual output of the network relative to the desired output.

From the "unrolled" figures above we see that the reccurent units are applied recursively for each word in the input sequence. This means the recurrent gate is applied once for each time-step. The gradient-signals have to flow back from the loss-function all the way to the first time the recurrent gate is used. If the gradient of the recurrent gate is multiplicative, then we essentially have an exponential function.

In this exercise, we will use texts that have more than 500 words. This means the RU's gate for updating its internal memory-state is applied recursively more than 500 times. If a gradient of just 1.01 is multiplied with itself 500 times then it gives a value of about 145. If a gradient of just 0.99 is multiplied with itself 500 times then it gives a value of about 0.007. These are called exploding and vanishing gradients. The only gradients that can survive recurrent multiplication are 0 and 1.

To avoid these so-called exploding and vanishing gradients, care must be made when designing the recurrent unit and its gates. That is why the actual implementation of the GRU is more complicated, because it tries to send the gradient back through the gates without this distortion.

## 3 - Imports

First, let's run the cell below to import all the packages that you will need during this exercise.

Run the following cell to load the packages and dependencies that you will find useful as you build the sentiment classifier!  We need to import several things from Keras.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 4 - Dataset

We will use a data-set consisting of 50000 reviews of movies from IMDB. Keras has a built-in function for downloading a similar data-set (but apparently half the size). However, Keras' version has already converted the text in the data-set to integer-tokens, which is a crucial part of working with natural languages that will also be demonstrated in this tutorial, so we use the donloaded dataset from Kaggle.



In [ ]:
df = pd.DataFrame()
df = pd.read_csv('IMDB Dataset.csv', encoding='utf-8')
df.head(3)

Now convert the class labels into integers.

In [ ]:
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df['sentiment'])
 
# printing label
label

In [ ]:
df.drop("sentiment", axis=1, inplace=True)
 
# Appending the array to our dataFrame
# with column name 'Purchased'
df["sentiment"] = label
 
# printing Dataframe
df

Load the training- and test-sets.

In [ ]:
x_train_text = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values
x_test_text = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
# Convert to numpy arrays.
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Combine into one data-set for some uses below.

In [ ]:
data_text = x_train_text + x_test_text

Print an example from the training-set to see that the data looks correct.

In [ ]:
# Print the first example from training set
### START CODE HERE ### 

### END CODE HERE ###

The true "class" is a sentiment of the movie-review. It is a value of 0.0 for a negative sentiment and 1.0 for a positive sentiment. Check if the sentiment for the above example is postive or negative.

In [ ]:
### START CODE HERE ### 

### END CODE HERE ###

## 5 - Data Pre-processing

### 5.1 - Tokenizer

A neural network cannot work directly on text-strings so we must convert it somehow. There are two steps in this conversion, the first step is called the "tokenizer" which converts words to integers and is done on the data-set before it is input to the neural network. The second step is an integrated part of the neural network itself and is called the "embedding"-layer, which is described further below.

We may instruct the tokenizer to only use e.g. the 10000 most popular words from the data-set. Call the function ***Tokenizer*** with the parameter as ***num_words***.

In [ ]:
num_words = 10000

In [ ]:
### START CODE HERE ### 
tokenizer = 
### END CODE HERE ###

The tokenizer can then be "fitted" to the data-set. This scans through all the text and strips it from unwanted characters such as punctuation, and also converts it to lower-case characters. The tokenizer then builds a vocabulary of all unique words along with various data-structures for accessing the data.

Note that we fit the tokenizer on the entire data-set so it gathers words from both the training- and test-data. This is OK as we are merely building a vocabulary and want it to be as complete as possible. The actual neural network will of course only be trained on the training-set.

Use `tokenizer.fit_on_texts` on whole data set

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

If you want to use the entire vocabulary then set `num_words=None` above, and then it will automatically be set to the vocabulary-size here. (This is because of Keras' somewhat awkward implementation.)

In [ ]:
if num_words is None:
    num_words = len(tokenizer.word_index)
print(len(tokenizer.word_index))

We can then inspect the vocabulary that has been gathered by the tokenizer. This is ordered by the number of occurrences of the words in the data-set. These integer-numbers are called word indices or "tokens" because they uniquely identify each word in the vocabulary.

In [ ]:
tokenizer.word_index

We can then use the tokenizer to convert all texts in the training-set to lists of these tokens.

In [ ]:
### START CODE HERE ###

# Use tokenizer.texts_to_sequences function on training set
x_train_tokens = 
### END CODE HERE ###

For example, here is a text from the training-set:

In [ ]:
x_train_text[0]

This text corresponds to the following list of tokens:

In [ ]:
np.array(x_train_tokens[0])

We also need to convert the texts in the test-set to tokens. Similarly do it for test data.

In [ ]:
### START CODE HERE ###
x_test_tokens = 
### END CODE HERE ###

### 5.2 - Padding and Truncating Data

The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

The average number of tokens in a sequence is:

In [ ]:
### START CODE HERE ###

# Use np.mean

### END CODE HERE ###


The maximum number of tokens in a sequence is:

In [ ]:
### START CODE HERE ###

### END CODE HERE ###


The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [ ]:
### START CODE HERE ###
max_tokens = 
### END CODE HERE ###


max_tokens = int(max_tokens)
max_tokens

This covers about 95% of the data-set.

In [ ]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

Use `pad_sequences` function with parameters as input, `maxlen`, `padding`, `truncating`


In [ ]:
pad = 'pre'

In [ ]:
### START CODE HERE ###
x_train_pad = 
### END CODE HERE ###


In [ ]:

### START CODE HERE ###
x_test_pad = 
### END CODE HERE ###


We have now transformed the training-set into one big matrix of integers (tokens) with this shape:

In [ ]:
x_train_pad.shape

The matrix for the test-set has the same shape:

In [ ]:
x_test_pad.shape

For example, we had the following sequence of tokens above:

In [ ]:
np.array(x_train_tokens[0])

This has simply been padded to create the following sequence. Note that when this is input to the Recurrent Neural Network, then it first inputs a lot of zeros. If we had padded 'post' then it would input the integer-tokens first and then a lot of zeros. This may confuse the Recurrent Neural Network.

In [ ]:
x_train_pad[0]

### 5.3 Tokenizer Inverse Map

For some strange reason, the Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. So we make that mapping here.

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

Helper-function for converting a list of tokens back to a string of words.

In [ ]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

For example, this is the original text from the data-set:

In [ ]:
x_train_text[0]

We can recreate this text except for punctuation and other symbols, by converting the list of tokens back to words:

In [ ]:
tokens_to_string(x_train_tokens[0])

## 6 - Create the Recurrent Neural Network

We are now ready to create the Recurrent Neural Network (RNN). We will use the Keras API for this because of its simplicity. 

In [ ]:
model = Sequential()

The first layer in the RNN is a so-called Embedding-layer which converts each integer-token into a vector of values. This is necessary because the integer-tokens may take on values between 0 and 10000 for a vocabulary of 10000 words. The RNN cannot work on values in such a wide range. The embedding-layer is trained as a part of the RNN and will learn to map words with similar semantic meanings to similar embedding-vectors, as will be shown further below.

First we define the size of the embedding-vector for each integer-token. In this case we have set it to 8, so that each integer-token will be converted to a vector of length 8. The values of the embedding-vector will generally fall roughly between -1.0 and 1.0, although they may exceed these values somewhat.

The size of the embedding-vector is typically selected between 100-300, but it seems to work reasonably well with small values for Sentiment Analysis.

In [ ]:
embedding_size = 8

The embedding-layer also needs to know the number of words in the vocabulary (`num_words`) and the length of the padded token-sequences (`max_tokens`). We also give this layer a name because we need to retrieve its weights further below.

In [ ]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

We can now add the first Gated Recurrent Unit (GRU) to the network. This will have 16 outputs. Because we will add a second GRU after this one, we need to return sequences of data because the next GRU expects sequences as its input.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###


This adds the second GRU with 8 output units. This will be followed by another GRU so it must also return sequences.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###


This adds the third and final GRU with 4 output units. This will be followed by a dense-layer, so it should only give the final output of the GRU and not a whole sequence of outputs. Then, add a fully-connected / dense layer which computes a value between 0.0 and 1.0 that will be used as the classification output.

**Note** Since it is a binary classification problem, so you have to use sigmoid activation function.

In [ ]:
### START CODE HERE ###   ≈2 lines of code


### END CODE HERE ###


Use the Adam optimizer with the given learning-rate.

In [ ]:
optimizer = Adam(lr=1e-3)

Compile the Keras model so it is ready for training. Use `binary_crossentrop` loss function because it s a binary classification problem.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

In [ ]:
model.summary()

### 6.1 - Train the Recurrent Neural Network

We can now train the model. Note that we are using the data-set with the padded sequences. We use 5% of the training-set as a small validation-set, so we have a rough idea whether the model is generalizing well or if it is perhaps over-fitting to the training-set. Input is x_train_pad, output is y_train and the number of epochs are set as 3 and batch_size is 64.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

### 6.2 - Performance on Test-Set

Now that the model has been trained we can calculate its classification accuracy on the test-set. Use `model_evaluate`function with input and output passed as parameters.

In [ ]:
### START CODE HERE ###
result = 
### END CODE HERE ###

In [ ]:
print("Accuracy: {0:.2%}".format(result[1]))

### 6.3 - Example of Mis-Classified Text

In order to show an example of mis-classified text, we first calculate the predicted sentiment for the first 1000 texts in the test-set.

In [ ]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

These predicted numbers fall between 0.0 and 1.0. We use a cutoff / threshold and say that all values above 0.5 are taken to be 1.0 and all values below 0.5 are taken to be 0.0. This gives us a predicted "class" of either 0.0 or 1.0.

In [ ]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

The true "class" for the first 1000 texts in the test-set are needed for comparison.

In [ ]:
cls_true = np.array(y_test[0:1000])

We can then get indices for all the texts that were incorrectly classified by comparing all the "classes" of these two arrays.

In [ ]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

Of the 1000 texts used, how many were mis-classified?

In [ ]:
len(incorrect)

Let us look at the first mis-classified text. We will use its index several times.

In [ ]:
idx = incorrect[0]
idx

The mis-classified text is:

In [ ]:
text = x_test_text[idx]
text

These are the predicted and true classes for the text:

In [ ]:
y_pred[idx]

In [ ]:
cls_true[idx]

## 7 - New Data

Let us try and classify new texts that we make up. Some of these are obvious, while others use negation and sarcasm to try and confuse the model into mis-classifying the text.

In [ ]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

We first convert these texts to arrays of integer-tokens because that is needed by the model.

In [ ]:
### START CODE HERE ###
tokens = 
### END CODE HERE ###


To input texts with different lengths into the model, we also need to pad and truncate them.

In [ ]:
### START CODE HERE ###
tokens_pad = 
### END CODE HERE ###

# Print the shape of the tokens_pad
### START CODE HERE ###

### END CODE HERE ###

We can now use the trained model to predict the sentiment for these texts.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

A value close to 0.0 means a negative sentiment and a value close to 1.0 means a positive sentiment. These numbers will vary every time you train the model.

## 8 - Embeddings

The model cannot work on integer-tokens directly, because they are integer values that may range between 0 and the number of words in our vocabulary, e.g. 10000. So we need to convert the integer-tokens into vectors of values that are roughly between -1.0 and 1.0 which can be used as input to a neural network.

This mapping from integer-tokens to real-valued vectors is also called an "embedding". It is essentially just a matrix where each row contains the vector-mapping of a single token. This means we can quickly lookup the mapping of each integer-token by simply using the token as an index into the matrix. The embeddings are learned along with the rest of the model during training.

Ideally the embedding would learn a mapping where words that are similar in meaning also have similar embedding-values. Let us investigate if that has happened here.

First we need to get the embedding-layer from the model:
Use `model.get_layer and pass the name of the layer as parameter

In [ ]:
### START CODE HERE ###
layer_embedding = 
### END CODE HERE ###


We can then get the weights used for the mapping done by the embedding-layer.
Use `layer_embedding.get_weights()` for a particular index.

In [ ]:
### START CODE HERE ###
weights_embedding = 
### END CODE HERE ###


Note that the weights are actually just a matrix with the number of words in the vocabulary times the vector length for each embedding. That's because it is basically just a lookup-matrix.

In [ ]:
weights_embedding.shape

Let us get the integer-token for the word 'good', which is just an index into the vocabulary.

In [ ]:
token_good = tokenizer.word_index['good']
token_good

Let us also get the integer-token for the word 'great'.

In [ ]:
token_great = tokenizer.word_index['great']
token_great

These integertokens may be far apart and will depend on the frequency of those words in the data-set.

Now let us compare the vector-embeddings for the words 'good' and 'great'. Several of these values are similar, although some values are quite different. Note that these values will change every time you train the model.

In [ ]:
weights_embedding[token_good]

In [ ]:
weights_embedding[token_great]

Similarly, we can compare the embeddings for the words 'bad' and 'horrible'.

In [ ]:
token_bad = tokenizer.word_index['bad']
token_horrible = tokenizer.word_index['horrible']

In [ ]:
weights_embedding[token_bad]

In [ ]:
weights_embedding[token_horrible]

### 8.1 - Sorted Words

We can also sort all the words in the vocabulary according to their "similarity" in the embedding-space. We want to see if words that have similar embedding-vectors also have similar meanings.

Similarity of embedding-vectors can be measured by different metrics, e.g. Euclidean distance or cosine distance.

We have a helper-function for calculating these distances and printing the words in sorted order.

In [ ]:
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

We can then print the words that are near and far from the word 'great' in terms of their vector-embeddings. Note that these may change each time you train the model.

In [ ]:
print_sorted_words('great', metric='cosine')

Similarly, we can print the words that are near and far from the word 'worst' in terms of their vector-embeddings.

In [ ]:
print_sorted_words('worst', metric='cosine')

## 9 - Conclusion

This exercise showed the basic methods for doing Natural Language Processing (NLP) using a Recurrent Neural Network with integer-tokens and an embedding layer. This was used to do sentiment analysis of movie reviews from IMDB. It works reasonably well if the hyper-parameters are chosen properly. But it is important to understand that this is not human-like comprehension of text. The system does not have any real understanding of the text. It is just a clever way of doing pattern-recognition.